In [ ]:
from autumn.tools.project import get_project
from matplotlib import pyplot
from autumn.tools.plots.utils import REF_DATE
# from autumn.calibration.targets import get_target_series
import pandas as pd
import json
from autumn.tools.utils.display import pretty_print
from autumn.settings import PROJECTS_PATH
import os
from autumn.tools.plots.uncertainty.plots import _get_target_values, _plot_targets_to_axis

In [ ]:
project = get_project("sm_sir", "national-capital-region")

In [ ]:
update_params = {
    'booster_effect_duration': 90.
}
params = project.param_set.baseline.update(update_params, calibration_format=True)


In [ ]:
model = project.run_baseline_model(params)
derived_df = model.get_derived_outputs_df()

In [ ]:
derived_df.plot.area(y=["prop_immune_low", "prop_immune_high"])

In [ ]:
outputs = ["hospital_occupancy", "icu_occupancy", "notifications", "prop_ever_infected"] #, "hospital_admissions", "hospital_occupancy", "icu_admissions", "icu_occupancy"]

if project.region_name == "national-capital-region":
    new_targets_path = os.path.join(PROJECTS_PATH, "sm_sir", "philippines", project.region_name, "new_targets.json")
    f = open(new_targets_path)
    new_targets = json.load(f)
    
for output in outputs:
    fig = pyplot.figure(figsize=(12, 8))
    pyplot.style.use("ggplot")
    axis = fig.add_subplot()

    axis = derived_df[output].plot()
    axis.set_title(output)
    
    if output in project.plots:
        if output.endswith("_occupancy") and project.region_name == "national-capital-region":
            values, times = new_targets[output]["values"], new_targets[output]["times"] 
        else:
            values, times = _get_target_values(project.plots, output)        
        
        date_times = pd.to_datetime(times, origin="01Jan2020",unit="D") 
        _plot_targets_to_axis(axis, values, date_times, on_uncertainty_plot=True)
         
